In [55]:
import pandas as pd
import matplotlib.pyplot as plt

In [118]:
counts = (pd.read_table('res/rrs.fuse.screen1.uniq.align.screen2.press.screen3.screen4.clust.shared')
            .drop(['label', 'numOtus'],
                  axis='columns')
            .rename(columns={'Group': 'library_id'})
            .set_index('library_id')
         )
#counts

taxonomy = (pd.read_table('res/rrs.fuse.screen1.uniq.align.screen2.press.screen3.screen4.clust.tax',
                          index_col='OTU')
              .Taxonomy
              .str.split('\([0-9]*\);')
              .apply(lambda x: pd.Series(x, index=['Domain', 'Phylum',
                                                   'Class', 'Order',
                                                   'Family', 'Genus',
                                                   'Species']))
              .drop('Species', axis='columns')
           )


counts.loc[counts.index.str.startswith('cult')]

,Otu001,Otu002,Otu003,Otu004,Otu005,Otu006,Otu007,Otu008,Otu009,Otu010,...,Otu895,Otu896,Otu897,Otu898,Otu899,Otu900,Otu901,Otu902,Otu903,Otu904
library_id,,,,,,,,,,,,,,,,,,,,,
cult-D0,0,3,528,855,0,0,0,1,1261,0,...,1,0,0,0,0,0,1,0,0,0
cult-D0-solids,0,90,653,1483,6,33,17,22,1324,3,...,0,0,0,0,0,0,0,0,0,0
cult-D1-1-6,1446,5,616,6664,1220,0,2,1,1791,1,...,0,0,0,0,0,0,0,0,0,0
cult-D1-1-7,1388,7,1578,6656,5666,2,0,7,2128,0,...,0,0,0,0,0,0,0,0,1,0
cult-D1-2-6,624,6,387,10064,424,0,1,0,1192,0,...,0,0,0,0,0,1,0,0,0,0
cult-D1-2-7,1256,0,1880,5979,4150,1,1,0,2185,0,...,0,1,1,0,1,0,0,0,0,0
cult-D6-1-6,8207,2,7414,1513,7641,0,0,2,2,0,...,0,0,0,0,0,0,0,0,0,0
cult-D6-1-7,6330,44,4181,7,2895,18,17,12,6,4,...,0,0,0,0,0,0,0,0,0,0
cult-D6-2-6,3437,1,2838,1804,1822,0,0,1,0,1,...,0,0,0,0,0,0,0,1,0,0


In [78]:
spike_hits = (pd.read_table('res/rrs.fuse.blastn-spike.tsv',
                            names=['seq_id', 'spike_id',
                                   'identity', 'align_length',
                                   'mismatch', 'gapopen', 'qstart', 'qend',
                                   'sstart', 'send', 'evalue', 'bitscore'],
                            index_col='seq_id')
                [['spike_id', 'identity', 'align_length']])


# Some of the S. alask sequences hits could actually be endemic sequences
spike_hits.drop(spike_hits[lambda x: (x.spike_id == 'sphingopyxis_alaskensis') &
                                     (x.identity < 98)].index,
                inplace=True)
# Some of the OTU-1 hits are not OTU-1
spike_hits.drop(spike_hits[lambda x: (x.spike_id == 'Otu00001') &
                                     ((x.identity < 98) | (x.align_length < 215))].index,
                inplace=True)



groups = (pd.read_table('res/rrs.fuse.groups',
                        names=['seq_id', 'library_id'], index_col='seq_id'))
library = pd.read_table('res/rrs.fuse.screen1.groups.tally.tsv',
                        names=['library_id', 'tally'], index_col='library_id')

In [79]:
_data = spike_hits.join(groups).groupby(['library_id', 'spike_id']).identity.count().unstack().join(library)
_data['endemic'] = _data.tally - _data.sphingopyxis_alaskensis

In [85]:
_data['ratio'] = _data.apply(lambda x: x.endemic / (x.spikeA + x.spikeB), axis=1)
_data

,Otu00001,sphingopyxis_alaskensis,spikeA,spikeB,tally,endemic,ratio
library_id,,,,,,,
cult-D0,205.0,524.0,4185.0,2769.0,6778,6254.0,0.899339
cult-D0-solids,240.0,630.0,3296.0,1974.0,9239,8609.0,1.633586
cult-D1-1-6,74.0,597.0,5941.0,3819.0,14837,14240.0,1.459016
cult-D1-1-7,294.0,930.0,5678.0,3404.0,26945,26015.0,2.864457
cult-D1-2-6,125.0,1129.0,6441.0,3979.0,17302,16173.0,1.552111
cult-D1-2-7,178.0,617.0,4423.0,2513.0,22452,21835.0,3.148068
cult-D6-1-6,NaN,1284.0,7033.0,4499.0,26338,25054.0,2.172563
cult-D6-1-7,413.0,565.0,4116.0,2551.0,17809,17244.0,2.586471
cult-D6-2-6,NaN,338.0,4961.0,2930.0,10240,9902.0,1.254847
